In [69]:
import gym
import random
import numpy as np
from keras.models     import Sequential
from keras.layers     import Dense
from keras.optimizers import Adam

In [75]:
env = gym.make('MountainCar-v0')
env.reset()
max_steps_count = 200
min_score_requirement = -198
training_games_count = 10000

In [79]:
def model_data_preparation():
    training_games_data = []
    accepted = []
    for i in range(training_games_count):
        score = 0
        memory = []
        prev_obs = []
        for step in range(max_steps_count):
            action = random.randrange(0, 2) * 2
            obs, reward, done, info = env.step(action)
            
            if len(prev_obs) > 0:
                memory.append([prev_obs, action])
                
            prev_obs = obs
            if obs[0] > -0.15:
                reward = 1
            score += reward
            if done:
                break
            
        if score >= min_score_requirement:
            accepted.append(score)
            for data in memory:
                if data[1] == 0:
                    output = (1, 0, 0)
                elif data[1] == 2:
                    output = (0, 0, 1)
                training_games_data.append((data[0], output))
        
        env.reset()
    
    
    return training_games_data

In [80]:
data = model_data_preparation()

In [81]:
def build_model(input_size, output_size):
        model = Sequential()
        model.add(Dense(128, input_dim=input_size, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(output_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam())

        return model

In [82]:
def train_model(training_data):
    x = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    model = build_model(input_size=len(x[0]), output_size=len(y[0]))
    
    model.fit(x, y, epochs=5)
    return model

In [83]:
trained_model = train_model(training_data)

Epoch 1/5
12935/12935 [==============================] - 2s 126us/step - loss: 0.1754
Epoch 2/5
12935/12935 [==============================] - 1s 56us/step - loss: 0.1652
Epoch 3/5
12935/12935 [==============================] - 1s 54us/step - loss: 0.1644
Epoch 4/5
12935/12935 [==============================] - 1s 61us/step - loss: 0.1637
Epoch 5/5
12935/12935 [==============================] - 1s 61us/step - loss: 0.1637


In [84]:
scores = []
for i in range(100):
    score = 0
    prev_obs = []
    for step in range(goal_steps):
#         env.render()
        if len(prev_obs) == 0:
            action = 0
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, len(prev_obs)))[0])
        
        prev_obs, reward, done, info = env.step(action)
        score += reward
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('average score:',sum(scores)/len(scores))

[-124.0, -126.0, -127.0, -132.0, -127.0, -127.0, -124.0, -120.0, -130.0, -138.0, -200.0, -119.0, -120.0, -121.0, -127.0, -200.0, -136.0, -128.0, -126.0, -133.0, -200.0, -119.0, -120.0, -121.0, -119.0, -200.0, -142.0, -123.0, -126.0, -126.0, -136.0, -130.0, -130.0, -200.0, -127.0, -127.0, -130.0, -130.0, -134.0, -123.0, -127.0, -121.0, -200.0, -121.0, -123.0, -119.0, -200.0, -125.0, -128.0, -200.0, -119.0, -121.0, -133.0, -126.0, -122.0, -119.0, -127.0, -128.0, -130.0, -139.0, -138.0, -200.0, -127.0, -127.0, -119.0, -127.0, -200.0, -119.0, -127.0, -127.0, -130.0, -200.0, -122.0, -126.0, -127.0, -119.0, -200.0, -122.0, -134.0, -200.0, -121.0, -126.0, -133.0, -120.0, -127.0, -200.0, -121.0, -127.0, -139.0, -130.0, -130.0, -127.0, -127.0, -130.0, -121.0, -123.0, -134.0, -123.0, -200.0, -120.0]
Average Score: -137.46
